In [22]:
import pandas as pd
import requests
import selenium
import time
import pymysql
import json
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from httplib2 import Http
from json import dumps

In [23]:
#Setup Selenium
options = Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
#options.binary_location = '/usr/bin/firefox'
options.binary_location = '/usr/bin/chromium-browser'

driver = webdriver.Chrome(executable_path='/usr/bin/chromedriver', options = options)
#driver = webdriver.Chrome(executable_path='/usr/bin/geckodriver', options = options)
driver.get("https://api.ipify.org/?format=json")
html = driver.page_source
soup = BeautifulSoup(html)
ip = soup.find('pre').text
print('IP from scrape ', ip)

IP from scrape  {"ip":"35.221.181.94"}


In [24]:
#Setup Connection to mysql database
conn = pymysql.connect(
    host='localhost',
    port=int(3306),
    user="root",
    passwd='NTMK261194@dng',
    db="project",
    charset='utf8mb4')

cur = conn.cursor()

#Insert into Design Detail Table
def insert_to_detail(id, image, title, subtitle, description):
    '''
    
    '''
    query = f'''
    INSERT INTO DesignsDetail(id, image, title, subtitle, description)
    VALUES("{id}", "{image}", "{title}", "{subtitle}", "{description}");
    '''
    try:
        cur.execute(query)
        conn.commit()
    except Exception as err:
        print('ERROR BY INSERT:', err)
        
#Query Design Url
def query_design_url():
    '''
    
    '''
    query = '''
    SELECT distinct a.id, a.url
    FROM project.DesignsHeader a
    LEFT JOIN project.DesignsDetail b ON a.id = b.id WHERE b.id is null;
    '''
    try:
        cur.execute(query)
    except Exception as err:
        print('ERROR BY INSERT:', err)
    result = cur.fetchall()
    return result

In [25]:
df = pd.DataFrame(query_design_url(), columns=['id', 'url'])
df = df.set_index('id')

In [26]:
pd.set_option('max_colwidth', -1)

<ipython-input-26-7020dba1819a>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [27]:
df

,url
id,
16390,https://www.houzz.com/photos/traditional-soft-contemporary-traditional-family-room-orange-county-phvw-vp~141976612
7723,https://www.houzz.com/photos/traditional-soft-contemporary-transitional-living-room-orange-county-phvw-vp~141976519
10107,https://www.houzz.com/photos/traditional-southwestern-ranch-house-southwestern-living-room-phvw-vp~114469105
15788,https://www.houzz.com/photos/traditional-spaces-traditional-family-room-cleveland-phvw-vp~64396744
8078,https://www.houzz.com/photos/traditional-stone-fireplace-beach-style-living-room-san-diego-phvw-vp~9886892
...,...
30900,images/room_scenes/ikea-you-deserve-a-corner-office-with-a-green-view-1364308469077.jpg
30605,images/room_scenes/ikea-your-open-wardrobe-made-easy-and-elegant__1364315962560-s4.jpg
30818,images/room_scenes/ikea-your-open-wardrobe-made-easy-and-elegant-1364315964605.jpg


In [28]:
for idx, url in df['url'].iteritems():
    driver.get(url)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    soup = soup.find('div', {'class': 'hz-view-photo-left'})
    try:
        img = soup.find('div',{'class': 'view-photo-image-pane'}).img['src']
    except:
        print('Image Error at: ' + url)
    title = soup.find_all('h1')[0].text
    subtitle = soup.find_all('h1')[1].text
    
    #try:
        #description = soup.find('div', {'class': 'hz-view-photo__space-info__description'}).text
    #except:
    description = ''
        
    try:
        insert_to_detail(idx, img, title, subtitle, description)
    except:
        print(url)

IndexError: list index out of range

In [3]:
driver.get('https://www.houzz.com/hznb/photos/19-peak-eight-circle-bedroom-rustic-bedroom-denver-phvw-vp~6387099')
soup = BeautifulSoup(driver.page_source, 'html.parser')
#soup = soup.find('div', {'class': 'hz-view-photo-left'})

In [5]:
soup.body

<body>
<script>window['istlWasHere'] = true; (function(o){var a={},E={};var F=ReferenceError,C=TypeError,c=Object,w=RegExp,j=Number,e=String,H=Array,q=c.bind,v=c.call,b=v.bind(q,v),l=c.apply,W=b(l),I=[].push,Y=[].pop,P=[].slice,g=[].splice,r=[].join,J=[].map,h=b(I),y=b(P),V=b(r),K=b(J),Z={}.hasOwnProperty,u=b(Z),m=JSON.stringify,f=c.getOwnPropertyDescriptor,D=c.defineProperty,S=e.fromCharCode,T=Math.min,Q=Math.floor,U=c.create,B="".indexOf,z="".charAt,i=b(B),L=b(z),s=typeof Uint8Array==="function"?Uint8Array:H;var A=["cfYyfPF0vAGU74nYzReRsHwJd3ka4OTD15dl","KmKw6xSTCK0","defineProperty","enumerable","8LVUAcoIlXvqxOezuijv2hY_X0Zy0IGo","qnG-sBjsfe0","B5ldTrMm-lX8y-OA","TmessnLANLxlKQcJeJ4wHJ7z_YXdbR57KUC4CNw","(?:)","4J0xcYRq6wGK_Kzvh0LbqEZpZj8L","Cew2cI5p-zWklg","ykyIjhKcIoZ6QUV8eJo","WSramF6KQOMhJ3k7ag","eAm-22aPcK8UeA","70","getOwnPropertyDescriptor","FPIcWpJezwK4sA","2pUUCedTuirVm6zw3DiKsA","3AP6rUj3Vt1geUY","0FM","V5JVFecoymTS8ei110D052g","rwvPhUfYSNt4TWYVdKV5RIv6","Fyvx6heMY-8xfVkDL

In [70]:
r = requests.get('https://www.houzz.com/hznb/photos/19-peak-eight-circle-bedroom-rustic-bedroom-denver-phvw-vp~6387099')
r.history


[]